In [ ]:
import os
import sys
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ✅ Ensure MODNet/src is in the Python path
modnet_path = "/Users/pamelasanchezhernandez/AfroVision/MODNet/src"
if modnet_path not in sys.path:
    sys.path.append(modnet_path)

# ✅ Set paths for pre-trained models
pretrained_path = "/Users/pamelasanchezhernandez/AfroVision/MODNet/pretrained/"
modnet_ckpt = os.path.join(pretrained_path, "modnet_photographic_portrait_matting.ckpt")
mobilenetv2_ckpt = os.path.join(pretrained_path, "mobilenetv2_backbone.ckpt")

# ✅ Check if models exist
if not os.path.exists(modnet_ckpt):
    raise FileNotFoundError(f"❌ MODNet checkpoint not found at {modnet_ckpt}")
if not os.path.exists(mobilenetv2_ckpt):
    raise FileNotFoundError(f"❌ MobileNetV2 backbone not found at {mobilenetv2_ckpt}")

print("✅ Pretrained models found! Loading MODNet...")

# ✅ Tell PyTorch where to find the models
os.environ["TORCH_HOME"] = pretrained_path

# ✅ Import MODNet
from models.modnet import MODNet

# ✅ Load MODNet Model with Correct Backbone
modnet = MODNet(backbone_pretrained=False)  # 🔹 Change to False to manually load
modnet.backbone.load_pretrained_ckpt = lambda: torch.load(mobilenetv2_ckpt, map_location=torch.device("cpu"))

modnet.load_state_dict(torch.load(modnet_ckpt, map_location=torch.device("cpu")), strict=False)
modnet.eval()  # Set model to evaluation mode

print("✅ MODNet loaded successfully!")

# ✅ Load and Preprocess Image
def preprocess_image(image_path):
    """Loads and preprocesses an image for MODNet."""
    image = Image.open(image_path).convert("RGB")
    image = image.resize((512, 512))  # Resize to model input size
    image = np.array(image).astype(np.float32) / 255.0  # Normalize
    image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0)  # Convert to tensor
    return image

# ✅ Update the image path if necessary
image_path = "/Users/pamelasanchezhernandez/AfroVision/sample.jpg"
input_image = preprocess_image(image_path)

# ✅ Apply MODNet Segmentation (Fix: Pass inference=True)
with torch.no_grad():
    matte = modnet(input_image, inference=True)[0]  # MODNet returns a list, so take the first item
    mask = matte[0].cpu().numpy()  # Extract mask

# ✅ Convert the Mask to a Binary Format
mask = (mask > 0.5).astype(np.uint8) * 255  # Thresholding

# ✅ Show Original and Masked Images
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(Image.open(image_path))
ax[0].set_title("Original Image")
ax[1].imshow(mask, cmap="gray")
ax[1].set_title("Curly Hair Segmentation Mask")
plt.show()


Updated Python Path: ['/opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/pamelasanchezhernandez/AfroVision/venv/lib/python3.11/site-packages', '/Users/pamelasanchezhernandez/AfroVision/MODNet/src']
✅ Pretrained models found! Loading MODNet...
cannot find the pretrained mobilenetv2 backbone


NameError: name 'exit' is not defined